In [1]:
from modules import read_from_allfiles
import re

docs = read_from_allfiles.doc_from_allfiles()
alltexts = ''.join(docs)
abbs = []
matches = re.finditer(r'\(([a-zA-Z\u0391-\u03c9][a-zA-Z\d\-–\/\u2215]+)\)', alltexts) #greek = re.compile(r'[\u0391-\u03c9]') # ギリシャ文字
# matches = re.finditer(r'\(([a-zA-Z\u0391-\u03c9][^\s\^\{\}\.\,\;\:\(\)\[\]]+?)s?\)', alltexts)
for match in matches:
    if not re.match(r'[a-z]+', match.group(1)):
        abbs.append(match.group(1))

abb_list = list(set(abbs)) 

100%|██████████| 17123/17123 [00:00<00:00, 22427.46it/s]


In [ ]:
# %%time
### df_aip2018, df_jap2005, df_aps2018 are obtained from MySQL server

from pathlib import Path
import re
import json
from tqdm import tqdm as tqdm
from gensim.models.phrases import Phrases , Phraser
from collections import Counter
from itertools import chain
import pickle


### wordを含む文を抽出
def ext_text_with_unit(word, doc):
    # word serach
    wsearch = re.compile(fr'[\002d\s](\({word}s?\))[\W]')

    ext_texts = []
    matches = re.finditer(wsearch, doc)
    if re.search(wsearch, doc):
        for m in matches:
            wis = m.start(1)
            wie = m.end(1)
            i = wis
            dis = wis
            while i > 0:
                if doc[i:i+1]=='.':
                    dis = i+1
                    break
                i -= 1
            j = wie
            die = wie
            while j < len(doc):
                if doc[j:j+1]=='.':
                    die = j
                    break
                j += 1
            ext_texts.append(doc[dis:die])
                   
    return ext_texts


## text処理 for phrases
def pre_pro_text(text):
    
    phrases = [] # gensim.phrases用
    
    re_blacs = re.compile(r'([\(\[“].+?[\)\]”])') # (phrase), [phrase]
    re_blac = re.compile(r'([\(\[“][^\s]+[\)\]”])') # (word), (“word”), [word] no spaces available inside blackets
    re_spases = re.compile(r'([\.\,\;\:]\s)') # . , ; : 

    # (word word) のカッコ内側にスペースを入れる (word)は除く
    blacs = re.finditer(re_blacs, text)
    for bl in blacs:        
        if not re.match(re_blac, bl.group(0)):  
            bs = bl.group(0).replace('(', '( ').replace(')', ' )').replace('[', '[ ').replace(']', ' ]')
            text = text.replace(bl.group(0), bs)  

    # [.,;:] + スペースの前にスペースを入れる（分割用）
    text = text.replace('. ', ' . ').replace(', ', ' , ').replace(': ', ' : ').replace('; ', ' ; ').replace('.\n', ' .\n')
    text = text.replace('.] ', ' . ] ').replace(',] ', ' , ] ').replace(':] ', ' : ] ').replace(';] ', ' ; ] ')
    text = text.replace('.\n', ' .\n').replace(',\n', ' ,\n').replace(':\n', ' :\n').replace(';\n', ' ;\n')

    # Phrases用コーパス        
    text = re.sub(r'\[[\d\,\-–\s]+\]', '', text) # []文献番号削除　[111]とかも消えるが多分問題ない？
    text = re.sub(r'\s[\.\,\;\:]\s', '\n', text) # スペース + [.,;:] + スペースで改行を入れる 記号をまたいだ複合語を作成しないため
    text = re.sub(r'\. \n', '\n', text) # 
    tlist = re.split(r'\n', text)
    for l in tlist:
        l = re.split(r'\s', l)
        if len(l) > 1:
            phrases.append(list(filter(lambda x: x != '', l)))

    return (phrases)

# bigram, trigram, quadtgram学習
def train_phrases(text_lists):
    mn = 1 # usually 5 
    th = 3

    bigram = Phrases(text_lists, min_count=mn, threshold=th)
    trigram = Phrases(bigram[text_lists], min_count=mn, threshold=th)
    quadgram = Phrases(trigram[bigram[text_lists]], min_count=mn, threshold=th)

    bigram_ = Phraser(bigram)
    trigram_ = Phraser(trigram)
    quadgram_ = Phraser(quadgram)
    
    return (bigram_, trigram_, quadgram_)


# bigram, trigram, quadtgram変換
def ext_phrases(train_data, texts):
    (bigram_, trigram_, quadgram_) = train_data
    bi_texts = bigram_[texts]
    tr_texts = trigram_[bi_texts]
    qd_texts = quadgram_[tr_texts]
    
    return qd_texts


ph_dic = {}
words = abb_list[0:100]

for word in tqdm(words):
    try:
        ext_texts = ext_text_with_unit(word, alltexts)
        ex_phrases = pre_pro_text(''.join(ext_texts))
        train_data = train_phrases(ex_phrases)
        phrases = ext_phrases(train_data, ex_phrases)
        all_phrases = list(chain.from_iterable(phrases))
        ex_phrases =  set([p for p in all_phrases if re.search(fr'_\({word}s?\)', p)])
        ph_dic[word] = ex_phrases
    except:
        print(word)

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
 97%|█████████▋| 97/100 [09:58<00:21,  7.31s/it]